# **(Crypto Currency Future Price Forecast ETL)**

## Objectives

Purpose of this ETL:
- prepare a clean, consistent dataset for multi-coin analysis and forecasting (top coins I chose: BTC, DOGE, ETH, HBAR, QNT, SOL, XDC, XLM, XRP).

What I will do
1) Load each coin CSV from `data/raw/`
2) Look at the head/shape/info so I understand what’s inside
3) Standardise column names (Date, Open, High, Low, Close, Volume), fix dtypes
4) Remove exact duplicates, handle missing values safely
5) Create helpful features (daily returns, log returns, 7/30-day moving averages, 30-day volatility)
6) Export tidy “long” table for the whole portfolio + per-coin processed CSVs
7) Save quick summary reports (coverage, recent volume, correlations) for my README/Power BI


## Inputs
- Raw Data: DataSet>Raw>BTC.csv
- Raw Data: DataSet>Raw>DOGE.csv
- Raw Data: DataSet>Raw>ETH.csv
- Raw Data: DataSet>Raw>HBAR.csv
- Raw Data: DataSet>Raw>QNT.csv
- Raw Data: DataSet>Raw>SOL.csv
- Raw Data: DataSet>Raw>XDC.csv
- Raw Data: DataSet>Raw>XLM.csv
- Raw Data: DataSet>Raw>XRP.csv

## Outputs
- Cleaned Data: DataSet>Cleaned>BTC_Processed.csv
- Cleaned Data: DataSet>Cleaned>DOGE_Processed.csv
- Cleaned Data: DataSet>Cleaned>ETH_Processed.csv
- Cleaned Data: DataSet>Cleaned>HBAR_Processed.csv
- Cleaned Data: DataSet>Cleaned>QNT_Processed.csv
- Cleaned Data: DataSet>Cleaned>SOL_Processed.csv
- Cleaned Data: DataSet>Cleaned>XDC_Processed.csv
- Cleaned Data: DataSet>Cleaned>XLM_Processed.csv
- Cleaned Data: DataSet>Cleaned>XRP_Processed.csv
- Cleaned Data: DataSet>Cleaned>All_Coins_Processed.csv



## Additional Comments
- This section was assisted by AI (ChatGPT-4) to help write a robust CSV loader function as I was encountering load errors due to inconsistent CSV formats from different sources. I provided the AI with examples of the different CSV formats and it generated a function that could handle these variations. I then reviewed and tested the function to ensure it worked correctly with my data.





---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\Nine\\OneDrive\\Documents\\VS Code Projects\\Crypto-Currency-Future-Price-Forecast\\jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\Nine\\OneDrive\\Documents\\VS Code Projects\\Crypto-Currency-Future-Price-Forecast'

# Section 1

Import libraries

In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

---

# Section 2

Load the raw data and start ETL process.

In [ ]:
# Used AI to help write this function as load errors were occurring
# due to inconsistent CSV formats from different sources.


def load_and_normalise_coin(symbol: str) -> pd.DataFrame:
    """
    Robust loader for coin CSVs with headers like: ticker,date,open,high,low,close
    If Volume is missing, creates Volume=0 so the rest of the pipeline works.
    Returns columns: ['Symbol','Date','Open','High','Low','Close','Volume']
    """
    path = DATA_RAW / f"{symbol}.csv"
    if not path.exists():
        raise FileNotFoundError(f"[{symbol}] Missing file: {path}")

    df = pd.read_csv(path, encoding_errors="ignore")

    # drop unnamed index columns if present
    for c in list(df.columns):
        if str(c).lower().startswith("unnamed"):
            df = df.drop(columns=[c])

    # normalise names
    cols = {c: str(c).strip().lower().replace(" ", "_") for c in df.columns}
    df = df.rename(columns=cols)

    # Detect columns
    # (Your files: ticker,date,open,high,low,close)
    date_col  = next((c for c in ["date","timestamp","time","datetime"] if c in df.columns), None)
    open_col  = next((c for c in ["open","open_price"] if c in df.columns), None)
    high_col  = "high" if "high" in df.columns else None
    low_col   = "low"  if "low"  in df.columns else None
    close_col = next((c for c in ["close","close_price","adj_close","adjusted_close"] if c in df.columns), None)

    if date_col is None or open_col is None or high_col is None or low_col is None or close_col is None:
        raise ValueError(f"[{symbol}] Required OHLC columns not found. Got: {list(df.columns)}")

    # Build output
    out = pd.DataFrame()
    out["Symbol"] = symbol

    # Date parsing (string dates assumed)
    out["Date"] = pd.to_datetime(df[date_col], errors="coerce")
    out = out.dropna(subset=["Date"])

    # Prices to numeric
    out["Open"]  = pd.to_numeric(df[open_col],  errors="coerce")
    out["High"]  = pd.to_numeric(df[high_col],  errors="coerce")
    out["Low"]   = pd.to_numeric(df[low_col],   errors="coerce")
    out["Close"] = pd.to_numeric(df[close_col], errors="coerce")

    # If volume doesn't exist, create it as 0
    if "volume" in df.columns:
        out["Volume"] = pd.to_numeric(df["volume"], errors="coerce").fillna(0)
    else:
        out["Volume"] = 0.0

    # Sort, de-dup, fill trivial OHLC gaps from Close when present
    out = out.sort_values("Date").drop_duplicates(subset=["Date"]).reset_index(drop=True)
    for col in ["Open","High","Low"]:
        mask = out[col].isna() & out["Close"].notna()
        out.loc[mask, col] = out.loc[mask, "Close"]

    # Drop rows where all OHLC are missing after cleanup
    keep_mask = ~out[["Open","High","Low","Close"]].isna().all(axis=1)
    out = out.loc[keep_mask].copy()

    return out[["Symbol","Date","Open","High","Low","Close","Volume"]]


In [14]:
# 
raw_frames, failed = [], []

for sym in COINS:
    try:
        df_sym = load_and_normalise_coin(sym)
        raw_frames.append(df_sym)
        print(f"[OK] {sym}: {df_sym.shape[0]} rows")
    except Exception as e:
        failed.append((sym, str(e)))
        print(f"[FAIL] {sym}: {e}")

if failed:
    print("\n Some symbols failed to load:")
    for s, err in failed:
        print(f" - {s}: {err}")

if len(raw_frames) == 0:
    raise RuntimeError("No coin files loaded — check the failures printed above.")

raw_all = pd.concat(raw_frames, axis=0, ignore_index=True)
print("\nCombined shape:", raw_all.shape)
display(raw_all.head())


[OK] BTC: 5521 rows
[OK] DOGE: 3345 rows
[OK] ETH: 3674 rows
[OK] HBAR: 2169 rows
[OK] QNT: 2396 rows
[OK] SOL: 1966 rows
[OK] XDC: 1813 rows
[OK] XLM: 3145 rows
[OK] XRP: 3869 rows

Combined shape: (27898, 7)


,Symbol,Date,Open,High,Low,Close,Volume
0,NaN,2010-07-17,0.04951,0.04951,0.04951,0.04951,0.0
1,NaN,2010-07-18,0.04951,0.08585,0.04951,0.08584,0.0
2,NaN,2010-07-19,0.08584,0.09307,0.07723,0.08080,0.0
3,NaN,2010-07-20,0.08080,0.08181,0.07426,0.07474,0.0
4,NaN,2010-07-21,0.07474,0.07921,0.06634,0.07921,0.0


In [15]:
# Health Check

print("Columns:", list(raw_all.columns))
raw_all.info()
print("\nMissing per column:\n", raw_all.isna().sum())

dup_rows = raw_all.duplicated(subset=["Symbol","Date"]).sum()
print("\nExact duplicates (by Symbol+Date):", dup_rows)
if dup_rows > 0:
    raw_all = raw_all.drop_duplicates(subset=["Symbol","Date"]).reset_index(drop=True)
    print("After dropping duplicates:", raw_all.shape)


Columns: ['Symbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27898 entries, 0 to 27897
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Symbol  0 non-null      object        
 1   Date    27898 non-null  datetime64[ns]
 2   Open    27898 non-null  float64       
 3   High    27898 non-null  float64       
 4   Low     27898 non-null  float64       
 5   Close   27898 non-null  float64       
 6   Volume  27898 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 1.5+ MB

Missing per column:
 Symbol    27898
Date          0
Open          0
High          0
Low           0
Close         0
Volume        0
dtype: int64

Exact duplicates (by Symbol+Date): 22377
After dropping duplicates: (5521, 7)


In [16]:
# Fill small gaps

def fill_small_gaps(group: pd.DataFrame, limit_days:int=2) -> pd.DataFrame:
    g = group.sort_values("Date").copy()
    g["Close_ff"] = g["Close"].ffill(limit=limit_days)
    for col in ["Open","High","Low"]:
        g[col] = g[col].fillna(g["Close_ff"])
    g["Close"] = g["Close"].ffill(limit=limit_days)
    # Volume is synthetic (0) for your files; keep it 0 or ffill small gaps if present
    g["Volume"] = g["Volume"].fillna(0)
    return g.drop(columns=["Close_ff"])

filled_all = raw_all.groupby("Symbol", group_keys=False).apply(fill_small_gaps)

In [19]:
import pandas as pd
import numpy as np

# --- fix the index/column clash for 'Symbol' ---

# 1) If 'Symbol' is also an index level, drop that index level without creating a new column
if getattr(filled_all.index, "names", None) and ("Symbol" in filled_all.index.names):
    filled_all = filled_all.reset_index(level="Symbol", drop=True)

# 2) If we somehow still have multiple 'Symbol' columns, keep the first and drop the rest
sym_cols = [i for i, c in enumerate(filled_all.columns) if c == "Symbol"]
if len(sym_cols) > 1:
    keep_idx = sym_cols[0]
    keep_mask = [True] * filled_all.shape[1]
    for i in sym_cols[1:]:
        keep_mask[i] = False
    filled_all = filled_all.loc[:, keep_mask]

# 3) Final sanity check
print("Index names:", filled_all.index.names)
print("Columns    :", list(filled_all.columns))
assert "Symbol" in filled_all.columns and ("Symbol" not in (filled_all.index.names or [])), "Symbol must be a plain column."

# --- feature engineering (same as before) ---
def add_features(group: pd.DataFrame) -> pd.DataFrame:
    g = group.sort_values("Date").copy()
    g["return_1d"]     = g["Close"].pct_change()
    g["log_return_1d"] = np.log(g["Close"]).diff()
    g["ma_7"]          = g["Close"].rolling(7,  min_periods=3).mean()
    g["ma_30"]         = g["Close"].rolling(30, min_periods=10).mean()
    g["vol_30d"]       = g["return_1d"].rolling(30, min_periods=10).std() * np.sqrt(365)
    return g

proc_all = (
    filled_all
    .groupby("Symbol", sort=False, group_keys=False)
    .apply(add_features)
    .reset_index(drop=True)
)

display(proc_all.head())


Index names: [None]
Columns    : ['Symbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']


,Symbol,Date,Open,High,Low,Close,Volume


In [20]:
combined_out = DATA_PROCESSED / "crypto_processed_long.csv"
proc_all.to_csv(combined_out, index=False)

for sym in COINS:
    proc_all[proc_all["Symbol"]==sym].to_csv(DATA_PROCESSED / f"{sym}_processed.csv", index=False)

print("Saved:", combined_out)


Saved: DataSet\Cleaned\crypto_processed_long.csv


In [21]:
# Duplicate check (Dataset said it was already cleaned but just in case)

dups = proc_all.duplicated(subset=["Symbol","Date"]).sum()
print("Exact duplicates (Symbol+Date):", dups)
# If >0:
if dups:
    proc_all = proc_all.drop_duplicates(subset=["Symbol","Date"]).reset_index(drop=True)


Exact duplicates (Symbol+Date): 0


---

---